In [1]:
import pandas as pd

# Load

In [40]:
def load_data():
    df = pd.read_csv(r"C:\Users\grego\Downloads\Checking1.csv", header=None)

    # give the columns names, and drop the columns we don't need
    df.columns = ["Date", "Amount", "DROP", "DROP2", "Description"]
    df.drop(["DROP", "DROP2"], axis=1, inplace=True)

    # convert the date column to datetime
    df["Date"] = pd.to_datetime(df["Date"])

    # sort the dataframe by date
    df.sort_values(by="Date", inplace=True)
    
    return df

In [41]:
df = load_data()
df.head()

,Date,Amount,Description
358,2023-01-03,-14.47,PURCHASE AUTHORIZED ON 12/30 STARBUCKS STORE 0...
357,2023-01-03,-26.88,PURCHASE AUTHORIZED ON 12/30 SQ *ACME GALLERY ...
356,2023-01-03,-27.62,PURCHASE AUTHORIZED ON 01/02 SUNOCO 80014559 P...
353,2023-01-04,-12.47,PURCHASE AUTHORIZED ON 01/04 HUDSONNEWS ST858 ...
355,2023-01-04,200.00,ONLINE TRANSFER FROM GLATZER G WAY2SAVE SAVING...


# Explore

In [61]:
def clean_description(df):
    """Call this function to clean the description column of the dataframe."""

    descr_clean = df.copy()

    # remove all of these patterns from the description column
    patterns = [
        r'PURCHASE AUTHORIZED ON \d{2}/\d{2}',
        r'\b\w{16}\sCARD\s\d{4}\b',
        r"PURCHASE INTL AUTHORIZED ON \d{2}/\d{2}"
    ]
    for pattern in patterns:
        descr_clean['Description'] = descr_clean['Description'].str.replace(pattern, '', regex=True).str.strip()

    return descr_clean

In [84]:
import plotly.express as px

def transactions_over_time(df):
    """Call this function to plot the transactions over time."""

    # clean the description column
    to_plot = clean_description(df)

    # Assign colors based on transaction values
    to_plot['Color'] = to_plot['Amount'].apply(lambda x: 'green' if x >= 0 else 'red')

    fig = px.bar(to_plot, x="Date", y="Amount", color="Color", title="Transactions Over Time", hover_data=["Description"], color_discrete_map={'green': 'green', 'red': 'red'})

    # make hover only show the description
    fig.update_traces(hovertemplate="<br>".join([
        "Date: %{x}",
        "Amount: %{y}",
        "Description: %{customdata[0]}"
    ]))

    # remove the legend
    fig.update_layout(showlegend=False)


    return fig

transactions_over_time(df).show()

In [101]:
def description_pie_chart(df, type="+", legend=False):
    """Call this function to plot the description pie chart, with the option to filter by type."""

    # clean the description column
    to_plot = clean_description(df)

    if type == "+":
        to_plot = to_plot[to_plot['Amount'] >= 0]
        title="Top 10 Positive Transaction Types"
    elif type == "-":
        to_plot = to_plot[to_plot['Amount'] < 0]
        title="Top 10 Negative Transaction Types"

    fig = px.pie(df, values='Description', names=df.index, title=title)

    # remove the legend
    fig.update_layout(showlegend=legend)

    return fig

description_pie_chart(df).show()

In [94]:
description_pie_chart(df, type="-", title="Top 10 Negative Transaction Types").show()

In [102]:
def clean_description(df):
    """Call this function to clean the description column of the dataframe."""

    descr_clean = df.copy()

    # remove all of these patterns from the description column
    patterns = [
        r"PURCHASE AUTHORIZED ON \d{2}/\d{2}",
        r"\b\w{16}\sCARD\s\d{4}\b",
        r"PURCHASE INTL AUTHORIZED ON \d{2}/\d{2}",
    ]
    for pattern in patterns:
        descr_clean["Description"] = (
            descr_clean["Description"].str.replace(pattern, "", regex=True).str.strip()
        )

    custom_categories = {
        "Penn State Apt Rent": "Rent",
        "ONLINE TRANSFER FROM GLATZER G": "Savings Transfer",
        "VENMO CASHOUT": "Venmo Cashout",
        "PANERA BREAD": "Panera Bread",
    }

    # replace the custom categories
    for custom_category in custom_categories:
        # if the custom category is in the description, replace it
        descr_clean.loc[
            descr_clean["Description"].str.contains(custom_category), "Description"
        ] = custom_categories[custom_category]

    return descr_clean

In [107]:
clean_description(df).groupby("Description").sum().reset_index().rename(columns={"Amount": "Total Amount"}).sort_values(by="Total Amount", ascending=False).head(10)

,Description,Total Amount
108,Savings Transfer,9100.00
156,Venmo Cashout,1146.36
42,IRS TREAS 310 TAX REF 032223 XXXXXXXXXX00908 G...,67.00
23,DUNKIN #349071 Q35 STATE COLLEGE PA,-1.39
106,SUNOCO 03679966 WAYNE PA,-1.49
65,RECURRING PAYMENT AUTHORIZED ON 03/13 GOOGLE *...,-2.11
68,RECURRING PAYMENT AUTHORIZED ON 05/12 GOOGLE *...,-2.11
67,RECURRING PAYMENT AUTHORIZED ON 04/12 GOOGLE *...,-2.11
88,SQ *MCLANAHAN'S DO State College PA,-2.22
34,GIANT 6072 255 NORTHLA STATE COLLEGE PA,-2.29
